In [2]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline

import torch.backends.cudnn as cudnn
cudnn.benchmark = True  
cudnn.deterministic = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print( device )
if torch.cuda.is_available():
    torch.cuda.caching_allocator_delete
    torch.cuda.empty_cache()  # 释放显存
    print('Memory Allocated', torch.cuda.memory_allocated() )

# 模型转到gpu： model.to(device)
# 训练数据转移到gpu上: x, y = x.to(device), y.to(device) ; 训练数据和test数据都都要


cuda:0
Memory Allocated 0


In [2]:
# torch.cuda.memory_stats()
# torch.cuda.memory_summary()
# torch.cuda.memory_allocated()

In [3]:
# 数据预处理
# df = pd.read_csv("../dataset/creditcard-tianchi.csv", index_col=None)
# del df['Time']
# df.head(3)
# df.head(30000).to_csv("../dataset/creditcard-3w.csv", index=False)  # 分拆数据到 creditcard-3w
# df.to_csv("../dataset/creditcard-all.csv", index=False)  # 分拆数据到 creditcard-3w

In [6]:
# 读取数据
df = pd.read_csv("../dataset/creditcard-all.csv", index_col=None)
# del df['Unnamed: 0']
df.head(3),df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 30 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V1      284807 non-null  float64
 1   V2      284807 non-null  float64
 2   V3      284807 non-null  float64
 3   V4      284807 non-null  float64
 4   V5      284807 non-null  float64
 5   V6      284807 non-null  float64
 6   V7      284807 non-null  float64
 7   V8      284807 non-null  float64
 8   V9      284807 non-null  float64
 9   V10     284807 non-null  float64
 10  V11     284807 non-null  float64
 11  V12     284807 non-null  float64
 12  V13     284807 non-null  float64
 13  V14     284807 non-null  float64
 14  V15     284807 non-null  float64
 15  V16     284807 non-null  float64
 16  V17     284807 non-null  float64
 17  V18     284807 non-null  float64
 18  V19     284807 non-null  float64
 19  V20     284807 non-null  float64
 20  V21     284807 non-null  float64
 21  V22     28

(         V1        V2        V3        V4        V5        V6        V7  \
 0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
 1  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
 2 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
 
          V8        V9       V10  ...       V21       V22       V23       V24  \
 0  0.098698  0.363787  0.090794  ... -0.018307  0.277838 -0.110474  0.066928   
 1  0.085102 -0.255425 -0.166974  ... -0.225775 -0.638672  0.101288 -0.339846   
 2  0.247676 -1.514654  0.207643  ...  0.247998  0.771679  0.909412 -0.689281   
 
         V25       V26       V27       V28  Amount  Class  
 0  0.128539 -0.189115  0.133558 -0.021053  149.62      0  
 1  0.167170  0.125895 -0.008983  0.014724    2.69      0  
 2 -0.327642 -0.139097 -0.055353 -0.059752  378.66      0  
 
 [3 rows x 30 columns],
 None)

In [7]:
df.Class.unique(), df.Class.value_counts()

(array([0, 1], dtype=int64),
 0    284315
 1       492
 Name: Class, dtype: int64)

In [8]:
# 列处理2
# df.groupby(['salary', 'sales']).size()

# pd.get_dummies(df.salary) # field内容编程列 list
# df = df.join( pd.get_dummies(df.salary) ) # 把列加入到 pandas fields
# del df['salary'] # 删除原来列

# pd.get_dummies(df.sales)  # 读日编码？？？
# df = df.rename(columns={'sales':'department'})  # 改列名
# df = df.join(pd.get_dummies(df.department))
# del df['department']
# df.head(3).to_csv("../dataset/test1.csv", index=False)

In [13]:

df.shape  # 行列？
df.size # df的行x列， 共有多少数据cell
df.Class.unique(), df.Class.value_counts()
# df.columns

(array([0, 1], dtype=int64),
 0    284315
 1       492
 Name: Class, dtype: int64)

In [16]:
Y_data = df.Class.values.reshape(-1,1)  # 转换成 pd type array
Y_data
# Y = torch.from_numpy(Y_data).type(torch.float32)  # 转换成tensor
# Y = torch.from_numpy(Y_data).type(torch.FloatTensor).to(device)
Y = torch.from_numpy(Y_data).type(torch.FloatTensor)
Y.size()

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [18]:
X_columns = [c for c in df.columns if c != 'Class']
X_data = df[X_columns]  
# X = torch.from_numpy(X_data.values).type(torch.float32).to(device)
X = torch.from_numpy(X_data.values).type(torch.float32)
X.shape, X.size()

(torch.Size([284807, 29]), torch.Size([284807, 29]))

In [19]:
import torch.nn.functional as F  # 避免relu和sigmoid的初始化，可以直接调用
class Model(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(29, 256)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(256, 128)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(128, 64)   # 接着，再定义64个后续中间层
        self.liner_4 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.relu( self.liner_3(x) )
        x = F.sigmoid( self.liner_4(x) )
        return x

# 更加高级语言法，等价于Model
class Model_high(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(20, 64)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(64, 64)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, input):
        x = self.Liner_1(input)
        x = self.relu(x)
        x = self.Liner_2(x)
        x = self.relu(x)
        x = self.Liner_3(x)
        x = self.sigmoid(x)
        return x

In [20]:
lr = 0.0001
def get_model():
    model = Model()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    return model, opt

In [21]:
model, optim = get_model()
model = model.to(device)
print( model, "\n" )
print( optim )
print(next(model.parameters()).device) # 应该打印 cuda:0

Model(
  (liner_1): Linear(in_features=29, out_features=256, bias=True)
  (liner_2): Linear(in_features=256, out_features=128, bias=True)
  (liner_3): Linear(in_features=128, out_features=64, bias=True)
  (liner_4): Linear(in_features=64, out_features=1, bias=True)
) 

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
cuda:0


In [22]:
loss_fn = nn.BCELoss()
# loss_fn = nn.BCELoss().to(device)
# batch = 64
# batch = 4096
# batch = 8192
# batch = 16384
batch = 32768 # 一次性加载
no_of_batches = len(X)//batch
epochs = 10
from torch.utils.data import TensorDataset, DataLoader


In [23]:
# Ver 4.1 - random_split Mode， charlie
# from torch.utils.data import random_split    
# def accuracy(y_pred, y_true):
#     y_pred = (y_pred > 0.5).type(torch.int32)
#     acc = (y_pred == y_true).float().mean()
#     return acc

# HR_dataset = TensorDataset(X, Y)
# train_ds, test_ds = random_split(HR_dataset, [0.8, 0.2]) 
# HR_dataloader = DataLoader(test_ds, batch_size=batch, shuffle=True)  # shuffle乱序取数
# epochMode = 41
# if epochMode == 41:  
#     for epoch in range( epochs ):
#         # for x,y in HR_dataloader:
#         for x,y in HR_dataloader:
#             y_pred = model(x)
#             loss = loss_fn(y_pred, y)
#             optim.zero_grad()
#             loss.backward()
#             optim.step()
#         with torch.no_grad():
#             print('epoch: ', epoch, ' loss: ', loss_fn(model(X), Y).data.item() )

In [24]:
# Ver 4.2 - sklearn mode
def accuracy(y_pred, y_true):
    y_pred = (y_pred > 0.5).type(torch.int32)
    acc = (y_pred == y_true).float().mean()
    return acc

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

train_x = torch.from_numpy(train_x.values).type(torch.float32).to(device)
train_y = torch.from_numpy(train_y).type(torch.float32).to(device)
train_ds = TensorDataset(train_x, train_y)
train_dl = DataLoader(train_ds, batch_size=batch, shuffle=True)  # 需要shuffle，乱序执行

test_x = torch.from_numpy(test_x.values).type(torch.float32).to(device)
test_y = torch.from_numpy(test_y).type(torch.float32).to(device)
test_ds = TensorDataset(test_x, test_y)
test_dl = DataLoader(test_ds, batch_size=batch, shuffle=False)  # 不需要shuffle，减少运算量

计算正确率
- Sigmoid()是0和1之间的值， 大于0.5为1，否则是0
- out1 = (y-pred >0.5).type(torch.int32)  : 转换成0或1
- (out1 == labels).float().mean()

In [25]:
%%time


# epochs = 10
for epoch in range( epochs ):
    for x,y in train_dl:
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
    with torch.no_grad():
        epoch_accuracy = accuracy(model(train_x), train_y)
        epoch_loss = loss_fn(model(train_x), train_y).data.item()
        epoch_test_accuracy = accuracy(model(test_x), test_y)
        epoch_test_loss = loss_fn(model(test_x), test_y).data.item()
        print('epoch: ', 
            epoch, ' train_loss: ', round(epoch_loss,4), ' Acc：', round(epoch_accuracy.item(),4),
            epoch, ' test_loss: ', round(epoch_test_loss,4), ' Acc：', round(epoch_test_accuracy.item(),4) )

epoch:  0  train_loss:  0.4843  Acc： 0.9838 0  test_loss:  0.4842  Acc： 0.9838
epoch:  1  train_loss:  0.4124  Acc： 0.9982 1  test_loss:  0.4124  Acc： 0.9983
epoch:  2  train_loss:  0.3655  Acc： 0.9982 2  test_loss:  0.3655  Acc： 0.9984
epoch:  3  train_loss:  0.3291  Acc： 0.9982 3  test_loss:  0.3293  Acc： 0.9984
epoch:  4  train_loss:  0.2964  Acc： 0.9982 4  test_loss:  0.2967  Acc： 0.9984
epoch:  5  train_loss:  0.2666  Acc： 0.9982 5  test_loss:  0.2671  Acc： 0.9984
epoch:  6  train_loss:  0.2397  Acc： 0.9982 6  test_loss:  0.2404  Acc： 0.9984
epoch:  7  train_loss:  0.2135  Acc： 0.9982 7  test_loss:  0.2139  Acc： 0.9984
epoch:  8  train_loss:  0.1884  Acc： 0.9982 8  test_loss:  0.1887  Acc： 0.9984
epoch:  9  train_loss:  0.1663  Acc： 0.9982 9  test_loss:  0.1664  Acc： 0.9984
CPU times: total: 16.1 s
Wall time: 29.7 s


In [26]:
# 最后一次计算结果：

epoch_accuracy = accuracy(model(train_x), train_y)
epoch_loss = loss_fn(model(train_x), train_y).data.item()
epoch_test_accuracy = accuracy(model(test_x), test_y)
epoch_test_loss = loss_fn(model(test_x), test_y).data.item()
print('epoch: ', 
    ' train_loss: ', round(epoch_loss,4), ' Acc：', round(epoch_accuracy.item(),4),
    ' test_loss: ', round(epoch_test_loss,4), ' Acc：', round(epoch_test_accuracy.item(),4) )

epoch:   train_loss:  0.1663  Acc： 0.9982  test_loss:  0.1664  Acc： 0.9984


In [27]:
# print( loss_fn(model(train_x), train_y).data.item() )
# accuracyRate = ((model(train_x).data.numpy()).astype('int') == train_y.numpy().astype('int')).mean()
# print("训练数据 - 准确率%：", accuracyRate*100) 

# print( loss_fn(model(test_x), test_y).data.item() )
# accuracyRate = ((model(test_x).data.numpy()).astype('int') == test_y.numpy().astype('int')).mean()
# print("测试数据 - 准确率%：", accuracyRate*100) 

- 数据准确性验证
- 过拟合/欠拟合
    - 过拟合：对于已知数据过度拟合，对未知数据预测性差
    - 欠拟合：对于已知数据拟合不够，对未知数据预测性差